### 라이브러리

In [25]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from dotenv import load_dotenv

### API 입력

In [ ]:
load_dotenv()

# API 키 설정
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_API_KEY    = os.getenv("OPENAI_API_KEY")

### 문서 로드/분할 및 벡터 임베딩

In [ ]:
# PDF 파일 로드 및 분할
loader = PyPDFLoader("KB 실버암 간편건강보험Plus.pdf")
pages = loader.load_and_split()

#PDF 파일을 300자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size    = 200, 
                                               chunk_overlap = 20,
                                               )
docs = text_splitter.split_documents(pages)

#텍스트 임베딩 (OpenAI 사용)
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small",
                              openai_api_key = OPENAI_API_KEY,
                              )

# 벡터 저장소 생성
vectorstore = FAISS.from_documents(docs, 
                                   embedding=embeddings,
                                   )
retriever = vectorstore.as_retriever()

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************L20A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

### 모델 선언

In [ ]:
# Claude LLM 선언
llm = ChatAnthropic(model = "claude-opus-4-20250514",
                    temperature = 0,
                    max_tokens = 1024,
                    api_key=ANTHROPIC_API_KEY,
                    )

# RAG 프롬프트 불러오기
prompt = hub.pull("rlm/rag-prompt")

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질문 및 답변
question = "c73이 림프절로 전이 되어 c77로 진단된 경우에는 어떤 질병으로 보니?"
answer = rag_chain.invoke(question)
print(answer)


C:\Users\KDT2\AppData\Roaming\Python\Python311\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


C73(갑상선의 악성신생물)이 림프절로 전이되어 C77(림프절의 이차성 및 상세불명의 악성신생물)로 진단된 경우에도 원발부위 기준에 따라 C73(갑상선의 악성신생물)에 해당하는 질병으로 봅니다.
